In [ ]:
import sys
from IPython.display import Image, display
import sympy

sys.path.append('..')   # path for local package
import restflow
from restflow import symvec

# Example 2: Neural Networks

We use the neural field equation developped by [Tiberi, 2022] given by:
\begin{equation*}
\partial_t\phi = \nabla^2(\kappa \phi + c_2 \phi^2 + c_3 \phi^3) + \eta
\end{equation*}
with non-conserved noise ($\alpha=0$).
By switching to Fourier space, the only non-zero vertices are:
\begin{equation*}
\cal{v}_2(q) = - c_2 q^2,
\end{equation*}
and 
\begin{equation*}
\cal{v}_3(q) = - c_3 q^2
\end{equation*}

<b><ins>1st Step:</b></ins> Define the model parameters, the vectors, the propagators and the vertex functions:

In [ ]:
# parameters of model
kap, c2, c3, D = sympy.symbols('kappa c2 c3 D')
d, Kd = sympy.symbols('d K_d')
# symbols for vectors
_q, _k, _p, dot_kq, dot_pk, dot_qp = sympy.symbols('q k p (k·q) (k·p) (q·p)')
_r, dot_qr, dot_pr, dot_rk = sympy.symbols('r (q·r) (p·r) (r·k)')

# assign symbol for dot product
symvec.dots[frozenset((_q,_k))] = dot_kq
symvec.dots[frozenset((_q,_p))] = dot_qp
symvec.dots[frozenset((_p,_k))] = dot_pk
symvec.dots[frozenset((_q,_r))] = dot_qr
symvec.dots[frozenset((_p,_r))] = dot_pr
symvec.dots[frozenset((_r,_k))] = dot_rk
# create vectors
k, q, p, r = symvec.Vector(_k), symvec.Vector(_q), symvec.Vector(_p), symvec.Vector(_r)

def f(x):
    return (kap*x**2)
# Important: Vertices should be tuples (numerator, denuminator)
def v2(k1,k2,k):
    expr = -c2*k**2
    return sympy.fraction(sympy.cancel(expr))

def v3(k1,k2,k3,k):
    return (-c3*k**2,1)

### The goal is to calculate the flow equations of the model parameters as shown in Eq. 53 and Eq. 54 of our manuscript.

# Propagator Corrections

The only one-loop graphs are the following ones:

In [ ]:
pil_img = Image(filename='./figures/modelB_plus_1vertex.jpg')
display(pil_img)

<b><ins>2nd Step:</b></ins> Create the graph using the `graph.py` script. Express the integrands and calculate the integrals using `integrals.symmetrize` and `integrals.solve` respectively:

In [ ]:
# graph (2c)
v = [restflow.Vertex() for i in range(3)]
v[0].link_vertex(v[1],0.0)
v[0].link_vertex(v[2],0.0)
v[2].link_vertex(v[1],0.12)
v[2].add_outgoing(0.0)
# express integrals of all graphs derived by symmetrized graph (output:array)
I_array = restflow.integrals.symmetrize(v, q, [q], f, D, k, v2, v3)
# solve each one of this integral and add them together
I2c = restflow.integrals.solve(I_array, k, [q], d, n=4)
# graph (3b)
v = [restflow.Vertex() for i in range(2)]
v[0].link_vertex(v[1],0.0)
v[0].link_vertex(v[1],0.12)
v[0].add_outgoing(0.0)
I_array = restflow.integrals.symmetrize(v, q, [q], f, D, k, v2, v3)
I3b = restflow.integrals.solve(I_array, k, [q], d, n=4)

Add the contribution of both diagrams:

In [ ]:
Iprop = I2c+I3b
display(Iprop)

# 2-Vertex Corrections

The only 1-loop diagrams contributing to 2-vertex are:

In [ ]:
pil_img = Image(filename='./figures/modelB_plus_2vertex.jpg')
display(pil_img)

We follow the same method for the 1-vertex corrections. There are two remarks:
-   The function `integrals.symmetrize` is necessary for the 2-vertices and 3-vertices because e.g. figures (b) and (d) represent multiple graphs (by permuting the labels of the external legs). 
-   We substitute all external wave vectors to zero before we solve the integral using `integrals.solve`.

In [ ]:
# graph (a)
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1], 0.0)
v[0].link_vertex(v[2], 0.12)
v[1].link_vertex(v[3], 0.0)
v[2].link_vertex(v[3], 0.0)
v[1].add_outgoing(-0.12)
v[2].add_outgoing(0.12)
I_array = restflow.integrals.symmetrize(v, q, [p,q-p], f, D, k,  v2, v3)
# substitute wavectors to zero
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), (element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]))) for element in I_array]
Ia = restflow.integrals.solve(I_array, k, [q, p], d, n=3)
# graph (b)
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1], 0.12)
v[0].link_vertex(v[2], 0.0)
v[2].link_vertex(v[3], 0.0)
v[3].link_vertex(v[1], 0.0)
v[2].add_outgoing(-0.12)
v[3].add_outgoing(-0.12)
I_array = restflow.integrals.symmetrize(v, q, [p,q-p], f, D, k,  v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), (element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]))) for element in I_array]
Ib = restflow.integrals.solve(I_array, k, [q, p], d, n=3)
# graph (c)
v = [restflow.Vertex() for i in range(3)]
v[0].link_vertex(v[1], 0.12)
v[0].link_vertex(v[2], 0.0)
v[2].link_vertex(v[1], 0.0)
v[2].add_outgoing(0.0)
v[2].add_outgoing(-0.12)
I_array = restflow.integrals.symmetrize(v, q, [p,q-p], f, D, k,  v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), (element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]))) for element in I_array]
Ic = restflow.integrals.solve(I_array, k, [q, p], d, n=3)
# graph (d)
v = [restflow.Vertex() for i in range(3)]
v[0].link_vertex(v[1], 0.12)
v[0].link_vertex(v[2], 0.0)
v[2].link_vertex(v[1], 0.0)
v[0].add_outgoing(-0.12)
v[2].add_outgoing(0.0)
I_array = restflow.integrals.symmetrize(v, q, [p,q-p], f, D, k,  v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), (element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]))) for element in I_array]
Id = restflow.integrals.solve(I_array, k, [q, p], d, n=3)

We sum the contribution of the integrals for the 2-vertex corrections:

In [ ]:
I2vert = Ia+Ib+Ic+Id
display(I2vert)

# 3-Vertex Corrections

The graphs which contribute to 3-vertex corrections are the following ones:

In [ ]:
pil_img = Image(filename='./figures/modelB_plus_3vertex.jpg')
display(pil_img)

We follow the same procedure as described above: 

In [ ]:
# figure (3c)
v = [restflow.Vertex() for i in range(3)]
v[0].link_vertex(v[1], 0.12)
v[0].link_vertex(v[2], 0.0)
v[2].link_vertex(v[1], 0.0)
v[0].add_outgoing(-0.12)
v[2].add_outgoing(0.12)
v[2].add_outgoing(-0.12)
I_array = restflow.integrals.symmetrize(v, q, [p,r,q-p-r], f, D, k,  v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), (element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]))) for element in I_array]
I3c = restflow.integrals.solve(I_array, k, [q, p], d, n=3)
# figure (e)
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1], 0.0)
v[1].link_vertex(v[2], 0.0)
v[0].link_vertex(v[3], 0.12)
v[3].link_vertex(v[2], 0.0)
v[0].add_outgoing(-0.12)
v[1].add_outgoing(-0.12)
v[3].add_outgoing(0.12)
I_array = restflow.integrals.symmetrize(v, q, [p,r,q-p-r], f, D, k,  v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), (element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]))) for element in I_array]
Ie = restflow.integrals.solve(I_array, k, [q, p], d, n=3)
# figure (f)
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1], 0.0)
v[0].link_vertex(v[2], 0.12)
v[2].link_vertex(v[3], 0.12)
v[3].link_vertex(v[1], 0.0)
v[0].add_outgoing(-0.12)
v[2].add_outgoing(0.0)
v[3].add_outgoing(-0.12)
I_array = restflow.integrals.symmetrize(v, q, [p,r,q-p-r], f, D, k,  v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), (element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]))) for element in I_array]
If = restflow.integrals.solve(I_array, k, [q, p], d, n=3)
# figure (g)
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1], 0.0)
v[1].link_vertex(v[2], 0.12)
v[0].link_vertex(v[3], 0.12)
v[3].link_vertex(v[2], 0.0)
v[1].add_outgoing(-0.12)
v[1].add_outgoing(-0.25)
v[3].add_outgoing(0.12)
I_array = restflow.integrals.symmetrize(v, q, [p,r,q-p-r], f, D, k,  v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), (element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]))) for element in I_array]
Ig = restflow.integrals.solve(I_array, k, [q, p], d, n=3)
# figure (h)
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1], 0.12)
v[0].link_vertex(v[2], 0.0)
v[2].link_vertex(v[3], 0.0)
v[3].link_vertex(v[1], 0.12)
v[3].add_outgoing(0.0)
v[3].add_outgoing(-0.12)
v[2].add_outgoing(-0.12)
I_array = restflow.integrals.symmetrize(v, q, [p,r,q-p-r], f, D, k,  v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), (element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]))) for element in I_array]
Ih = restflow.integrals.solve(I_array, k, [q, p], d, n=3)
# figure (i)
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1], 0.12)
v[0].link_vertex(v[2], 0.0)
v[2].link_vertex(v[3], 0.0)
v[3].link_vertex(v[1], 0.12)
v[2].add_outgoing(-0.12)
v[2].add_outgoing(0.12)
v[3].add_outgoing(-0.12)
I_array = restflow.integrals.symmetrize(v, q, [p,r,q-p-r], f, D, k,  v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), (element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]))) for element in I_array]
Ii = restflow.integrals.solve(I_array, k, [q, p], d, n=3)
# figure (j)
v = [restflow.Vertex() for i in range(5)]
v[0].link_vertex(v[1], 0.0)
v[1].link_vertex(v[2], 0.0)
v[2].link_vertex(v[3], 0.0)
v[0].link_vertex(v[4], 0.12)
v[4].link_vertex(v[3], 0.0)
v[1].add_outgoing(-0.12)
v[2].add_outgoing(-0.12)
v[4].add_outgoing(0.12)
I_array = restflow.integrals.symmetrize(v, q, [p,r,q-p-r], f, D, k,  v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), (element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]))) for element in I_array]
Ij = restflow.integrals.solve(I_array, k, [q, p], d, n=3)
# figure (k)
v = [restflow.Vertex() for i in range(5)]
v[0].link_vertex(v[1], 0.12)
v[0].link_vertex(v[2], 0.0)
v[2].link_vertex(v[3], 0.0)
v[3].link_vertex(v[4], 0.0)
v[4].link_vertex(v[1], 0.0)
v[2].add_outgoing(-0.12)
v[3].add_outgoing(-0.12)
v[4].add_outgoing(-0.12)
I_array = restflow.integrals.symmetrize(v, q, [p,r,q-r-p], f, D, k,  v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), (element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]))) for element in I_array]
Ik = restflow.integrals.solve(I_array, k, [q, p], d, n=3)

We sum all of the contributions to 3-vertex corrections:

In [ ]:
I3vert = sympy.Poly(Ie+If+Ig+Ih+Ii+Ij+Ik+I3c,q.sym).as_expr()
display(I3vert)

# Extracting the Corrections of the Model Parameters

We just calculated the corrections to the <b>propagator, 2-vertex and 3-vertex</b>.
<br> Now, we can extract the corrections to the <b>model parameters</b>. For this, we need to:
-   Define new renormalized vertices (with new model parameters)
-   Solve linear equations derived by comparing the coefficients of the monomials of the original vertices with those of the renormalized vertices (see Appendix C of our manuscript)

In [ ]:
kapt, c1t, c2t, c3t, Dt = sympy.symbols('kappat c1t c2t c3t Dt')

def v2t(k1,k2,k):
    expr = -c2t*k**2
    return sympy.fraction(sympy.cancel(expr))

def v3t(k1,k2,k3,k):
    return (-c3t*k**2,1)

def compare_coeffs(dict1,dict_original,k,q,p):
    """
    Returns equations between the renormalized model parameters and the original model parameters
    """
    eqts = [0]*len(dict1)
    i=0
    for key in dict1:
        eqts[i]= dict1.get(key) - dict_original.get(key)
        i+=1
    eqts = [i for i in eqts if i != 0]
    return eqts

def expand_vertex(expr, q):
    """
    Expands the coefficients of the renormalized 2-vertex
    Returns dictionary of the coefficients of the monomials
    """
    return symvec.all_coeffs(expr,[q.sym])

def renormalize_vertex(expr,vertex,k,q,p,d):
    """
    Matches the coefficients of the 2-vertex with the renormalized 2-vertex.
    """
    coeffs = expand_vertex(expr,q)
    coeffs_original = expand_vertex(vertex[0]/vertex[1],q)
    eqts = compare_coeffs(coeffs,coeffs_original,k,q,p)
    if vertex == v2t(q,q,q):
        return list(sympy.linsolve(eqts, [c2t]))[0]
    elif vertex == v3t(q,q,q,q):
        return list(sympy.linsolve(eqts, [c3t]))[0]

For the $\phi_\kappa$ we use the following relationship:
\begin{equation*}
\frac{1}{\widetilde{f}(q)}=\frac{1}{f(q)}+I, 
\end{equation*}
and by taylor expanding, we get:
\begin{equation*}
\widetilde{f}(k)\approx f(k)-I, 
\end{equation*}

Using the corrections to the propagators, we get that $\psi_\kappa$ is:

In [ ]:
from IPython.display import Latex
display(Latex('$\psi_\kappa=$'),sympy.simplify(-Iprop/(kap*q**2)))

Similarly, we get for the 2-vertex and 3-vertex:

In [ ]:
from IPython.display import Latex
# renormalize c2 parameter
sol_c2t = renormalize_vertex(I2vert,v2t(q,q,q),k,q,p,d)[0]
psi_2=sympy.simplify(sol_c2t/c2)
display(Latex('$\psi_2=$'),psi_2)
# renormalize c3 parameter
sol_c3t = renormalize_vertex(I3vert,v3t(q,q,q,q),k,q,p,d)[0]
psi_3=sympy.simplify(sol_c3t/c3)
display(Latex('$\psi_3=$'),psi_3)
